## Sentiment Analysis

### Sentiment Analysis of tweets from Ireland (@Agriculture_ie)

In [ ]:
# Import Libraries

import os
import pandas as pd
from textblob import TextBlob
import numpy as np 
import re 
import matplotlib.pyplot as plt

import sys
import tweepy
import nltk

import pycountry
import string
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

In [ ]:
consumer_key = "P8SkXDMp6yC2ztHyN4gFky3Eg"
consumer_secret = "qrV4S8mBDDGXWZboErzGu3zVpx00l3O6JcqMMJCbF9NXxZLrfC"
access_token = "1055465543308849153-OXE8Rc2JH3UEmcj1Nob5F7gHwymA1n"
access_token_secret = "a9gp8qzhfzrRItEdMk6q0fHw2EvLrp9c0AqNbxmSsUIFx"

In [ ]:
%run ./keys.ipynb

In [ ]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [ ]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

keyword = "@agriculture_ie"
noOfTweet = 500
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
 
 #print(tweet.text)
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity

    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
 
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1
    
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [ ]:
#Number of Tweets (Total, Positive, Negative, Neutral)

tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

In [ ]:
#Creating PieCart

labels = ['Positive ['+str(positive)+'%]' , 'Neutral ['+str(neutral)+'%]','Negative ['+str(negative)+'%]']
sizes = [positive, neutral, negative]
colors = ['yellowgreen', 'blue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use("default")
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= "+keyword+"" )
plt.axis("equal")
plt.show()

In [ ]:
tweet_list.head()

In [ ]:
tweet_list.shape

In [ ]:
tweet_df = tweet_list.rename(columns={tweet_list.columns[0]: 'tweet'})

In [ ]:
tweet_df.head(3)

In [ ]:
tweet_df['word_count'] = tweet_df['tweet'].apply(lambda x: len(str(x).split(" ")))

In [ ]:
tweet_df.head(3)

In [ ]:
tweet_df['char_count'] = tweet_df['tweet'].str.len()

In [ ]:
## this also includes spaces
tweet_df[['tweet','char_count']].head()

In [ ]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

In [ ]:
tweet_df['avg_word'] = tweet_df['tweet'].apply(lambda x: avg_word(x))

In [ ]:
tweet_df[['tweet','avg_word']].head()

In [ ]:
stop = stopwords.words('english')

In [ ]:
tweet_df['stopwords'] = tweet_df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
tweet_df[['tweet','stopwords']].head()

In [ ]:
tweet_df['hastags'] = tweet_df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
tweet_df[['tweet','hastags']].head()

In [ ]:
tweet_df['numerics'] = tweet_df['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
tweet_df[['tweet','numerics']].head()

In [ ]:
tweet_df['upper'] = tweet_df['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
tweet_df[['tweet','upper']].head()

In [ ]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
tweet_df['tweet'].head()

In [ ]:
tweet_df['tweet'] = tweet_df['tweet'].str.replace('[^\w\s]','')
tweet_df['tweet'].head()

In [ ]:
tweet_df['tweet'] = tweet_df['tweet'].str.replace('5','')
tweet_df['tweet'].head()

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
tweet_df['tweet'].head()

In [ ]:
freq = pd.Series(' '.join(tweet_df['tweet']).split()).value_counts()[:10]
freq

In [ ]:
freq = list(freq.index)
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
tweet_df['tweet'].head()

In [ ]:
from textblob import TextBlob
tweet_df['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
nltk.download('punkt')
TextBlob(tweet_df['tweet'][1]).words

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
tweet_df['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
nltk.download('wordnet')
from textblob import Word
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
tweet_df['tweet'].head()

In [ ]:
TextBlob(tweet_df['tweet'][0]).ngrams(2)

In [ ]:
tf1 = (tweet_df['tweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

In [ ]:
import numpy as np

for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(tweet_df.shape[0]/(len(tweet_df[tweet_df['tweet'].str.contains(word)])))

tf1

In [ ]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(tweet_df['tweet'])

train_vect

In [ ]:
print(train_vect)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(tweet_df['tweet'])
train_bow

In [ ]:
print(train_bow)

In [ ]:
tweet_df['tweet'][:5].apply(lambda x: TextBlob(x).sentiment)

In [ ]:
tweet_df['sentiment'] = tweet_df['tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
tweet_df[['tweet','sentiment']].head()

In [ ]:
tweet_df.head(5)

In [ ]:
tweet_df = tweet_df[["tweet"]]

In [ ]:
tweet_df.to_csv('sentiment.txt')

In [ ]:
df = pd.read_csv("sentiment.txt")

In [ ]:
import os
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
text = "Big Data Agriculture"

x, y = np.ogrid[:300, :300]

mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)


wc = WordCloud(background_color="white", repeat=True, mask=mask)
wc.generate(text)

plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()

In [ ]:
# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# Read the whole text.
text = open(path.join(d, 'sentiment.txt')).read()

# read the mask image
# taken from
# http://www.stencilry.org/stencils/movies/alice%20in%20wonderland/255fk.jpg
alice_mask = np.array(Image.open(path.join(d, "Ireland_flag.png")))

stopwords = set(STOPWORDS)
stopwords.add("said")

wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
               stopwords=stopwords, contour_width=3, contour_color='steelblue')

# generate word cloud
wc.generate(text)

# store to file
wc.to_file(path.join(d, "alice.png"))

# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# lower max_font_size, change the maximum number of word and lighten the background:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Save the image in the img folder:
wordcloud.to_file("first_review.png")

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["drink", "now", "wine", "flavor", "flavors"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# import pillow as PIL